In [1]:
def extract_size_from_config(content):
    
    size_match = re.search(r'Size=(\d+)\s+(\d+)', content)
    if size_match:
        columns = int(size_match.group(1))
        rows = int(size_match.group(2))
        print(f"Extracted Size: Columns={columns}, Rows={rows}")
        return columns, rows
    else:
        raise ValueError("Size not found in the configuration file")

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import json
import shutil
import rasterio
import re
np.seterr(divide='ignore', invalid='ignore')

index_angle = [1, 8, 9, 10, 11, 12, 13, 25, 26, 27, 28, 29]  #

pathsim = "C:/Users/moham/DART/user_data/simulations/Simulation_monastir5_2parc/sequence"  # l'emplacement des sequences
pathsavesim = "C:/Users/moham/OneDrive/Bureau/summer_internship/data/grp2/4"
lst = os.listdir(pathsim)

nbreseq = len(lst)

props_dict = dict()

for dir in lst:
    savedirectory = pathsavesim + dir
    if os.path.exists(savedirectory):
        shutil.rmtree(savedirectory)
    os.makedirs(savedirectory)

    g = open(pathsim + '/' + dir + '/output/dart.sequenceur.properties', 'r')
    contenu = g.read()
    caractere = "\n"
    x2 = contenu.split(caractere)
    
    for k in range(1, len(x2) - 1, 10):
        for i in range(0,10,2):
            prop = (x2[k+i].split(":")[1]).split(".")[-1]
            prop_value = float(x2[k +i + 1].split(":")[1])
            props_dict[prop + str((k // 10))] = prop_value

    with open(savedirectory + "/props.json", "w") as outfile:
        json.dump(props_dict, outfile)
    
    bands = ["BAND0","BAND1","BAND2","BAND3","BAND4","BAND5","BAND6","BAND7","BAND8","BAND9"]
    bands_arr = []
    
    for band in bands:
        band_folder = pathsim + '/' + dir + '/output/' + band + '/BRF/ITERX/IMAGES_DART'
        ima_prefixed = [filename for filename in os.listdir(band_folder) if filename.startswith("ima")]
        header_name = [filename for filename in ima_prefixed if filename.endswith("mpr")][0]

        with open(os.path.join(band_folder, header_name), "rb") as header_file:
            header_content = header_file.read().decode()  # Read and decode the content
            rows,columns = extract_size_from_config(header_content)
        
        header_file = open(os.path.join(band_folder, header_name), "rb")
        header_data = header_file.read()
        
        img_name = [filename for filename in ima_prefixed if filename.endswith("mp#")][0]
        img_data = np.fromfile(os.path.join(band_folder, img_name), dtype=np.double)
        print(band,' ',min(img_data),' ',max(img_data))
        img_data = np.reshape(img_data, (rows, columns))
        #plt.figure()
        #plt.imshow(img_data, cmap='gray')
        #plt.title(band)
        #plt.show()
        
        img_data_16bit = (10000 * img_data).astype(np.uint16)
        bands_arr.append(img_data_16bit)
    
    img_data_16bit_all_bands = np.array(bands_arr)
    imagename = savedirectory + "/" + dir + ".tif"
    
    with rasterio.open(imagename, 'w', driver='GTiff', height=img_data_16bit.shape[0], width=img_data_16bit.shape[1],
                       count=len(bands_arr), dtype=str(img_data_16bit.dtype)) as dst:
        for i in range(len(bands_arr)):
            dst.write(bands_arr[i], i + 1)

    
    src = rasterio.open(imagename)
    array=src.read()
    print(dir)
    '''
    for band in array:
        plt.figure()
        plt.imshow(band, cmap='gray')
        #plt.title(band)
        plt.show()
    '''
    print(np.max(array - img_data_16bit_all_bands))


Extracted Size: Columns=48, Rows=82
BAND0   0.01064796734921916   0.07390944270062769
Extracted Size: Columns=48, Rows=82
BAND1   0.02047488278424108   0.14963501312192415
Extracted Size: Columns=48, Rows=82
BAND2   0.012823581837911368   0.23851617451990315
Extracted Size: Columns=48, Rows=82
BAND3   0.02931949418241422   0.2674962439610068
Extracted Size: Columns=48, Rows=82
BAND4   0.11258810335609298   0.3380689120546674
Extracted Size: Columns=48, Rows=82
BAND5   0.13548284392048568   0.4035379405741819
Extracted Size: Columns=48, Rows=82
BAND6   0.1419429496852122   0.4133907851697063
Extracted Size: Columns=48, Rows=82
BAND7   0.1427671943825024   0.4141404561506001
Extracted Size: Columns=48, Rows=82
BAND8   0.0771945227723948   0.5074264071252825
Extracted Size: Columns=48, Rows=82
BAND9   0.02390587115274735   0.4824832894027758
allproperty_0
0
Extracted Size: Columns=48, Rows=82
BAND0   0.010625993386654549   0.0738974036202603
Extracted Size: Columns=48, Rows=82
BAND1   0.0

C:\Users\moham\AppData\Local\Programs\Python\Python311\Lib\site-packages\rasterio\__init__.py:327: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
C:\Users\moham\AppData\Local\Programs\Python\Python311\Lib\site-packages\rasterio\__init__.py:317: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


BAND6   0.1304290006813884   0.4118507495388568
Extracted Size: Columns=48, Rows=82
BAND7   0.13109266293179114   0.41351644164276485
Extracted Size: Columns=48, Rows=82
BAND8   0.07367257872462503   0.5098777015608946
Extracted Size: Columns=48, Rows=82
BAND9   0.024434784334446395   0.4842578975254353
allproperty_1
0
Extracted Size: Columns=48, Rows=82
BAND0   0.010904047934461125   0.07389791563549895
Extracted Size: Columns=48, Rows=82
BAND1   0.0238834619568853   0.14964581666766313
Extracted Size: Columns=48, Rows=82
BAND2   0.015064713229140863   0.23861640217243932
Extracted Size: Columns=48, Rows=82
BAND3   0.03591723828234768   0.2675464170497263
Extracted Size: Columns=48, Rows=82
BAND4   0.12981767421910886   0.34600601894247046
Extracted Size: Columns=48, Rows=82
BAND5   0.15930725894102846   0.3967417132039093
Extracted Size: Columns=48, Rows=82
BAND6   0.16581086232433034   0.4105038299556167
Extracted Size: Columns=48, Rows=82
BAND7   0.16669927006399246   0.41232125873

In [3]:
g.close()
header_file.close()
src.close()